# Updated functions

`update` removes incidents from the table that have an endtime 5 hours before the current time.

`insert_data` now removes duplicate incidents before appending remaining incidents to the table.

In [ ]:
from datetime import datetime, timedelta

def update(conn):
    """Updates incidents table by removing incidents with end times five hours before current time"""
    cutoffTime = datetime.now() + timedelta(hours=5)
    cursor = conn.cursor()
    cmd=\
    f"""
    DELETE FROM incidents
    WHERE endDatetime < '{cutoffTime.strftime('%Y-%m-%d %H:%M:%S')}'
    """
    cursor.execute(cmd)

def insert_data(conn, data):
    """Inserts new traffic data into incidents table in database"""

    if len(data) > 0:
        data['endDatetime'] = data['endTime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))
        data.to_sql("intermediate", conn, if_exists="append", index=False)
        cursor = conn.cursor()
        cmd=\
        """
        INSERT INTO incidents
        SELECT * from intermediate
        WHERE id NOT IN (SELECT id FROM intermediate)
        """
        cursor.execute(cmd)
        cmd = "DROP TABLE intermediate"
        cursor.execute(cmd)